# EDA on the BigBasket Case
## Exploratory Data Analysis (EDA)
- Ci Zhu
- Jan 2020
- For: MMAI 831

This Notebook will load in each file in the dataset and perform some basic EDA and summary statistics on it.

In [1]:
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Load in the data files

In [2]:
# Import Data
df = pd.read_csv('Big Basket.csv')

In [3]:
## Take a look at the data
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62141 entries, 0 to 62140
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Member       62141 non-null  object
 1   Order        62141 non-null  int64 
 2   SKU          62141 non-null  int64 
 3   Created On   62141 non-null  object
 4   Description  62141 non-null  object
dtypes: int64(2), object(3)
memory usage: 2.4+ MB


,Member,Order,SKU,Created On,Description
0,M09736,6468572,34993740,22-09-2014 22:45,Other Sauces
1,M09736,6468572,15669800,22-09-2014 22:45,Cashews
2,M09736,6468572,34989501,22-09-2014 22:45,Other Dals
3,M09736,6468572,7572303,22-09-2014 22:45,Namkeen
4,M09736,6468572,15669856,22-09-2014 22:45,Sugar


# Data Cleaning

In [4]:
## Notice that the "Created On" is supposed to be a date type but now an object type

## we need to:
## 1. split the datetime from date
## 2. clean date format

# 1. split the datetime from date
df[['Created On Date','Created On Time']] = df['Created On'].str.split(' ',expand=True,)
df

,Member,Order,SKU,Created On,Description,Created On Date,Created On Time
0,M09736,6468572,34993740,22-09-2014 22:45,Other Sauces,22-09-2014,22:45
1,M09736,6468572,15669800,22-09-2014 22:45,Cashews,22-09-2014,22:45
2,M09736,6468572,34989501,22-09-2014 22:45,Other Dals,22-09-2014,22:45
3,M09736,6468572,7572303,22-09-2014 22:45,Namkeen,22-09-2014,22:45
4,M09736,6468572,15669856,22-09-2014 22:45,Sugar,22-09-2014,22:45
...,...,...,...,...,...,...,...
62136,M64379,8381435,15670260,16-11-2013 00:43,Organic F&V,16-11-2013,00:43
62137,M64379,8381435,15668597,16-11-2013 00:43,Exotic Vegetables,16-11-2013,00:43
62138,M64379,8381435,7570555,16-11-2013 00:43,Shoe Polish,16-11-2013,00:43
62139,M64379,8381435,7587490,16-11-2013 00:43,Organic Dals & Pulses,16-11-2013,00:43


In [5]:
## 2. clean date format
## The length of the standard datetime format is 10 including the '-'
## filter out all the observations that has a shorter length and convert them to standard format

## create the helper function
def clean_date (str):
    first_slash = str.find('/')
    second_slash = str.find('/',3)
    day = ('0' + str[:first_slash])[-2:]
    month = ('0' + str[first_slash + 1:second_slash])[-2:]
    year = str[-4:]
    return day + '-' + month + '-' +  year

## test
## clean_date ('1/2/2020')


df['Created On Date'] = df['Created On Date'].map(clean_date) 

## validate
violation = df['Created On Date'].str.len() < 10
df.loc[violation]


df

,Member,Order,SKU,Created On,Description,Created On Date,Created On Time


,Member,Order,SKU,Created On,Description,Created On Date,Created On Time
0,M09736,6468572,34993740,22-09-2014 22:45,Other Sauces,01-01-2014,22:45
1,M09736,6468572,15669800,22-09-2014 22:45,Cashews,01-01-2014,22:45
2,M09736,6468572,34989501,22-09-2014 22:45,Other Dals,01-01-2014,22:45
3,M09736,6468572,7572303,22-09-2014 22:45,Namkeen,01-01-2014,22:45
4,M09736,6468572,15669856,22-09-2014 22:45,Sugar,01-01-2014,22:45
...,...,...,...,...,...,...,...
62136,M64379,8381435,15670260,16-11-2013 00:43,Organic F&V,01-01-2013,00:43
62137,M64379,8381435,15668597,16-11-2013 00:43,Exotic Vegetables,01-01-2013,00:43
62138,M64379,8381435,7570555,16-11-2013 00:43,Shoe Polish,01-01-2013,00:43
62139,M64379,8381435,7587490,16-11-2013 00:43,Organic Dals & Pulses,01-01-2013,00:43


# Data Mining

In [6]:
# Extract some info about the date

df['Created_On_Date'] = pd.to_datetime(df['Created On Date'])
df['Created_On_DOW'] = df['Created_On_Date'].dt.day_name()
df['Created_On_Week'] = df['Created_On_Date'].dt.strftime('%U')
df['Created_On_Month'] = df['Created_On_Date'].dt.month_name()
df['Created_On_IsWeekend'] = np.where(df['Created_On_Date'].dt.weekday < 5, 0, 1)


## validate if we created unclean column

df[df.isnull().any(axis=1)]



,Member,Order,SKU,Created On,Description,Created On Date,Created On Time,Created_On_Date,Created_On_DOW,Created_On_Week,Created_On_Month,Created_On_IsWeekend


In [7]:
# Create TimeSlice from 'Created On Time'

def Timeslicer_Number(x):
    if (x>=0) & (x<6):
        Timeslice = 0
    elif (x>=6) & (x<12):
        Timeslice = 1
    elif (x>=12) & (x<18):
        Timeslice = 2
    else:
        Timeslice = 3
    return Timeslice

def Timeslicer_Name(x):
    if (x>=0) & (x<6):
        Timeslice = 'Mid-Night'
    elif (x>=6) & (x<12):
        Timeslice = 'Morning'
    elif (x>=12) & (x<18):
        Timeslice = 'Afternoon'
    else:
        Timeslice = 'Evening'
    return Timeslice



df['Created_On_Time'] = pd.to_datetime(df['Created On Time'])
df['Create_On_Timeslice_Number'] = df['Created_On_Time'].dt.hour.map(Timeslicer_Number) 
df['Create_On_Timeslice_Name'] = df['Created_On_Time'].dt.hour.map(Timeslicer_Name) 

#del df ['Created On Time']
#del df ['Created_On_Time']

df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62141 entries, 0 to 62140
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Member                      62141 non-null  object        
 1   Order                       62141 non-null  int64         
 2   SKU                         62141 non-null  int64         
 3   Created On                  62141 non-null  object        
 4   Description                 62141 non-null  object        
 5   Created On Date             62141 non-null  object        
 6   Created On Time             62141 non-null  object        
 7   Created_On_Date             62141 non-null  datetime64[ns]
 8   Created_On_DOW              62141 non-null  object        
 9   Created_On_Week             62141 non-null  object        
 10  Created_On_Month            62141 non-null  object        
 11  Created_On_IsWeekend        62141 non-null  int32     

,Member,Order,SKU,Created On,Description,Created On Date,Created On Time,Created_On_Date,Created_On_DOW,Created_On_Week,Created_On_Month,Created_On_IsWeekend,Created_On_Time,Create_On_Timeslice_Number,Create_On_Timeslice_Name
0,M09736,6468572,34993740,22-09-2014 22:45,Other Sauces,01-01-2014,22:45,2014-01-01,Wednesday,00,January,0,2021-01-25 22:45:00,3,Evening
1,M09736,6468572,15669800,22-09-2014 22:45,Cashews,01-01-2014,22:45,2014-01-01,Wednesday,00,January,0,2021-01-25 22:45:00,3,Evening
2,M09736,6468572,34989501,22-09-2014 22:45,Other Dals,01-01-2014,22:45,2014-01-01,Wednesday,00,January,0,2021-01-25 22:45:00,3,Evening
3,M09736,6468572,7572303,22-09-2014 22:45,Namkeen,01-01-2014,22:45,2014-01-01,Wednesday,00,January,0,2021-01-25 22:45:00,3,Evening
4,M09736,6468572,15669856,22-09-2014 22:45,Sugar,01-01-2014,22:45,2014-01-01,Wednesday,00,January,0,2021-01-25 22:45:00,3,Evening
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62136,M64379,8381435,15670260,16-11-2013 00:43,Organic F&V,01-01-2013,00:43,2013-01-01,Tuesday,00,January,0,2021-01-25 00:43:00,0,Mid-Night
62137,M64379,8381435,15668597,16-11-2013 00:43,Exotic Vegetables,01-01-2013,00:43,2013-01-01,Tuesday,00,January,0,2021-01-25 00:43:00,0,Mid-Night
62138,M64379,8381435,7570555,16-11-2013 00:43,Shoe Polish,01-01-2013,00:43,2013-01-01,Tuesday,00,January,0,2021-01-25 00:43:00,0,Mid-Night
62139,M64379,8381435,7587490,16-11-2013 00:43,Organic Dals & Pulses,01-01-2013,00:43,2013-01-01,Tuesday,00,January,0,2021-01-25 00:43:00,0,Mid-Night


In [8]:
## Aggregate by SKU to explore the popular items and rare items


SKU_groups = df.groupby(['SKU']).agg(
    {
        'SKU': ['max', 'count'],
        'Description':['max']
        ##'Category': ['max'],
    }).reset_index()


SKU_groups.columns = ["_".join(x) for x in SKU_groups.columns.ravel()]
SKU_groups = SKU_groups.rename(columns={'SKU_': "SKU",
                                         'SKU_count': 'count'})

# Let's sort the item_groups by the ones that appear the most
SKU_groups = SKU_groups.sort_values('count', ascending=False)

In [9]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.float_format', lambda x: '%.1f' % x):
    display(SKU_groups.head(30))

,SKU,SKU_max,count,Description_max
683,15668381,15668381,1702,Other Vegetables
737,15668688,15668688,1540,Root Vegetables
698,15668460,15668460,1439,Gourd & Cucumber
681,15668379,15668379,1415,Other Vegetables
716,15668478,15668478,1252,Banana
706,15668468,15668468,1137,Beans
680,15668378,15668378,1003,Other Vegetables
705,15668467,15668467,963,Beans
686,15668416,15668416,936,Banana
703,15668465,15668465,923,Root Vegetables


In [10]:
# Define my own version of the 'mode' function, which will just return the first mode (in case of tie) and thus avoid an annoying error from groupby (whcih always wants just one value)
my_mode = lambda x: pd.Series.mode(x)[0]

members_order = df.groupby('Member').agg(
    {
        'Created_On_Date': ['count', 'min', 'max'],
        'Created_On_Week': [my_mode],
        'Create_On_Timeslice_Number': ['mean', 'max', my_mode],
        
        'SKU': ['count', my_mode],        
        'Order': ['nunique'],
        'Description':[my_mode]   
}).reset_index()

members_order.columns = ["_".join(x) for x in members_order.columns.ravel()]
members_order = members_order.rename(columns={'Member_': "Member"})
members_order = members_order.rename(columns={'Order_nunique': "Total Orders"})
members_order.columns = members_order.columns.str.replace("<lambda_0>", "mode")
members_order.columns = members_order.columns.str.replace("<lambda>", "mode")

# Let's sort the members by the ones that have attended the most
#members_order = members_order.sort_values('Order', ascending=False)

In [11]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.float_format', lambda x: '%.1f' % x):
    display(members_order.head(30))

,Member,Created_On_Date_count,Created_On_Date_min,Created_On_Date_max,Created_On_Week_mode,Create_On_Timeslice_Number_mean,Create_On_Timeslice_Number_max,Create_On_Timeslice_Number_mode,SKU_count,SKU_mode,Total Orders,Description_mode
0,M04158,493,2012-01-01,2014-12-04,00,2.1,3,2,493,15668378,132,Other Vegetables
1,M08075,638,2012-01-01,2014-12-02,00,1.6,3,1,638,15668465,55,Beans
2,M09303,463,2013-01-01,2014-12-08,00,2.1,3,3,463,15668478,105,Namkeen
3,M09736,626,2013-01-01,2014-12-04,00,2.1,3,3,626,15668478,62,Banana
4,M12050,556,2012-01-01,2014-08-02,00,2.4,3,3,556,15668471,39,Organic F&V
5,M12127,431,2012-01-01,2014-05-01,00,1.8,3,1,431,15670260,38,Organic F&V
6,M14746,647,2013-01-01,2014-12-05,00,1.9,3,3,647,15668381,71,Organic F&V
7,M16218,554,2013-01-01,2014-12-07,00,1.8,3,1,554,15668416,92,Other Vegetables
8,M16611,420,2013-01-01,2014-12-04,00,2.3,3,3,420,15668467,62,Namkeen
9,M18732,432,2013-01-01,2014-12-01,00,1.8,3,1,432,15668458,79,Other Vegetables
